In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', 'af1030deb58b4162abd1eec5ab6ac81f')

In [ ]:
results = search_images_bing(key, 'grizzly bear')
ims = results.attrgot('contentUrl')
len(ims)

In [ ]:
ims = ['http://3.bp.blogspot.com/-S1scRCkI3vY/UHzV2kucsPI/AAAAAAAAA-k/YQ5UzHEm9Ss/s1600/Grizzly%2BBear%2BWildlife.jpg']

In [ ]:
dest = 'images/grizzly.jpg'
download_url(ims[0], dest)

In [ ]:
im = Image.open(dest)
im.to_thumb(128,128)

In [ ]:
bear_types = 'grizzly','black','teddy'
path = Path('bears')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in bear_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} bear')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
# our folder has image files
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

In [ ]:
# Independent varaible - ImageBlock (x)
# Independent variable is the thing we are using to make predictions from
# Dependent variable - CategoryBlock (y)
# Dependent variable is our target
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files, # tell how to get image files
    splitter=RandomSplitter(valid_pct=0.2, seed=42), # seed starting point keep seed same to get everytime sane random number to generate same validation set everytime
    get_y=parent_label, # how to create label in dataset
    item_tfms=Resize(128) # to keep all images of same size use Transform - Resize
)


In [ ]:
# now we have datablock object - bears, which is template for creeating data loaders
# tell data loaders where are our data e.g. actual images
dls = bears.dataloaders(path)

# data loaders includes - training and validation data loaders.
# what data loaders do actually ?
# it will give mini batch at a time to the GPU, loop through it and it will give you 64 images of batch at a time in single tensor.

# to see what batch contains
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
# create learner and fine tune
learn = cnn_learner(dls, resenet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
#confusion matrix to see how manny images are classified incorrectly
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)